In [122]:
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [123]:
x = np.linspace(0, 10, 100)

y1 = np.sin(np.pi * x)
y2 = np.cos(2 * np.pi * x)
y3 = np.sin(7 * np.pi * x)

y = y1 + y2 + y3

fig = make_subplots(rows=2,
                    cols=3,
                    specs=[
                        [{}, {}, {}],
                        [{'colspan': 3}, None, None]
                    ],
                    subplot_titles=(
                        'sin(&#120529;n)',
                        'cos(2&#120529;n)',
                        'sin(7&#120529;n)',
                        'SUM')
                    )  # &#120529; is html code for pi

fig.add_trace(go.Scattergl(x=x, y=y1, mode='lines'), row=1, col=1)
fig.add_trace(go.Scattergl(x=x, y=y2, mode='lines'), row=1, col=2)
fig.add_trace(go.Scattergl(x=x, y=y3, mode='lines'), row=1, col=3)
fig.add_trace(go.Scattergl(x=x, y=y, marker=dict(color='black'), mode='markers'), row=2, col=1)

fig.update_layout(showlegend=False, height=700)
fig.update_xaxes(title='n', row=2, col=1)
fig.update_yaxes(title='X', row=1, col=1)
fig.update_yaxes(title='X', row=2, col=1)

fig.show()

In [132]:
transform_ex = np.fft.rfft(y, axis=0)

sr = len(x)/max(x)
N = transform_ex.size
n = np.arange(N)
T = N/sr
freq_ex = n/T

# freq_ex = np.fft.rfftfreq(n, timestep)

fig = go.Figure()
fig.add_trace(go.Scatter(x=freq_ex, y=transform_ex.real, marker=dict(color='blue'), name='Real', mode='lines'))
fig.add_trace(go.Scatter(x=freq_ex, y=transform_ex.imag, marker=dict(color='red'), name='Imaginary', mode='lines'))

# fig.update_xaxes(range=[0, 10], title='k')
fig.update_yaxes(title='FFT(X)')
fig.update_xaxes(range=[0, 10])
fig.update_layout(height=700)

fig.show()

In [125]:
def sine_calculation(freqs: list, x: float) -> float:
    result = 0

    for frequency in freqs:
        result += np.sin(frequency * np.pi * x)

    return result


def cosine_calculation(freqs: list, x: float) -> float:
    result = 0

    for frequency in freqs:
        result += np.cos(frequency * np.pi * x)

    return result

In [126]:
frequency_bound = max(np.quantile(transform_ex.real, q=0.99), np.quantile(transform_ex.imag, q=0.99))

iter_size = len(transform_ex)
sine_freqs = list()
cosine_freqs = list()

for idx in range(0, iter_size):
    if np.abs(transform_ex[idx].real) >= frequency_bound:
        cosine_freqs.append(freq_ex[idx])
    if np.abs(transform_ex[idx].imag) >= frequency_bound:
        sine_freqs.append(freq_ex[idx])

In [127]:
sine_freqs, cosine_freqs

([0.9803921568627452], [1.9607843137254903])

In [128]:
result = list()
for idx in np.linspace(10, 20, 100):
    result.append(sine_calculation(sine_freqs, idx) + cosine_calculation(cosine_freqs, idx))

In [130]:
import plotly.express as px

x_test = np.linspace(10, 20, 100)
prediction = result
actual = np.sin(np.pi * x_test) + np.cos(2 * np.pi * x) + np.sin(7 * np.pi * x)

fig = px.line(x=x_test, y=[actual, result])
fig.update_xaxes(range=[10, 15])
fig.show()